**Run the following two cells before you begin.**

In [1]:
%autosave 10

Autosaving every 10 seconds


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [5]:
# Import the data set
cleaned_data=pd.read_csv("https://raw.githubusercontent.com/jsneha1710/Data-Science-Intern-2-Mini-Project-Task/master/Task%201%20Data%20set/cleaned_data.csv")


In [30]:
# Define the sigmoid function
def sig(x):
    s=1/(1+np.exp(-x))
    return s


**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [14]:
# Create a train/test split
features=['PAY_1','LIMIT_BAL']
X=cleaned_data[features]
y=cleaned_data['default payment next month']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=24)



______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [15]:
logreg = LogisticRegression(solver='liblinear')

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [16]:
# Fit the logistic regression model on training data

logreg.fit(X_train, y_train)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
# Make predictions using `.predict()`
y_pred = logreg.predict(X_test)



In [18]:
# Find class probabilities using `.predict_proba()`
y_pred_proba = logreg.predict_proba(X_test)[::,1]
y_pred_proba


array([0.25173076, 0.415703  , 0.20395547, ..., 0.415703  , 0.17278502,
       0.33606565])

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [42]:
cleaned_data.shape

(26664, 30)

In [43]:
# Add column of 1s to features
ones=[]
i=0
for i in range(26664):
    ones.append(1)
cleaned_data['ones']=ones
cleaned_data.ones


0        1
1        1
2        1
3        1
4        1
5        1
6        1
7        1
8        1
9        1
10       1
11       1
12       1
13       1
14       1
15       1
16       1
17       1
18       1
19       1
20       1
21       1
22       1
23       1
24       1
25       1
26       1
27       1
28       1
29       1
        ..
26634    1
26635    1
26636    1
26637    1
26638    1
26639    1
26640    1
26641    1
26642    1
26643    1
26644    1
26645    1
26646    1
26647    1
26648    1
26649    1
26650    1
26651    1
26652    1
26653    1
26654    1
26655    1
26656    1
26657    1
26658    1
26659    1
26660    1
26661    1
26662    1
26663    1
Name: ones, Length: 26664, dtype: int64

In [23]:
# Get coefficients and intercepts from trained model
print(logreg.coef_)
print(logreg.intercept_)


[[ 8.27451187e-11 -6.80876727e-06]]
[-6.57647457e-11]


In [61]:
# Manually calculate predicted probabilities
p=(8.27451187e-11)*cleaned_data['PAY_1']+(-6.80876727e-06)*cleaned_data['LIMIT_BAL']+(-6.57647457e-11)*cleaned_data['ones']
sig(p)

0        0.466009
1        0.306390
2        0.351423
3        0.415703
4        0.415703
5        0.415703
6        0.032159
7        0.336066
8        0.278236
9        0.466009
10       0.203955
11       0.145508
12       0.013526
13       0.383052
14       0.154179
15       0.415703
16       0.466009
17       0.079354
18       0.226956
19       0.292114
20       0.306390
21       0.383052
22       0.044619
23       0.351423
24       0.415703
25       0.399266
26       0.415703
27       0.172785
28       0.415703
29       0.336066
           ...   
26634    0.264771
26635    0.466009
26636    0.203955
26637    0.079354
26638    0.367095
26639    0.215233
26640    0.415703
26641    0.432330
26642    0.054183
26643    0.108059
26644    0.226956
26645    0.415703
26646    0.415703
26647    0.351423
26648    0.466009
26649    0.163269
26650    0.079354
26651    0.292114
26652    0.154179
26653    0.264771
26654    0.278236
26655    0.193123
26656    0.482985
26657    0.336066
26658    0

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [75]:
# Manually calculate predicted classes
m_pred=[]
for i in sig(p):
    if(i >= 0.5):
        m_pred.append(1)
    else:
        m_pred.append(0)
m_pred




[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [38]:
# Compare to scikit-learn's predicted classes
y_pred


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [39]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
from sklearn import metrics
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
auc


0.627207450280691

In [77]:
# Use manually calculated predicted probabilities to calculate ROC AUC
from sklearn import metrics
try:
    auc2=metrics.roc_auc_score(m_pred,sig(p))
    auc2
except ValueError:
    pass

